In [1]:
file_name = "graph_path_planning.ipynb"
import sys
import os
sys.path.insert(0, os.path.abspath(
    os.path.join(os.path.dirname(file_name), '../../src/') ))

import numpy as np

from utils import *

In [2]:
clientID = connect_2_sim()
test_connection(clientID)

Connected to remote API server
Number of objects in the scene:  42


In [3]:
# Recuperar handlers do dr20
left_motor_handle, right_motor_handle = get_dr20_motor_handles_(clientID)

# getting sensor handle
sensor_reference = './dr20/ultrasonicSensorJoint_/ultrasonicSensor_'
err_code,ultras1 = sim.simxGetObjectHandle(clientID=clientID,
                                            objectName=sensor_reference,
                                            operationMode=sim.simx_opmode_blocking)

motor handles successfully retrieved!


In [19]:
from IPython.display import clear_output

while True:
    clear_output(wait=True)
    _, _, detected_point, _, _ = sim.simxReadProximitySensor(clientID=clientID, sensorHandle=ultras1, operationMode=sim.simx_opmode_blocking)
    distance = np.sqrt(detected_point[0]**2 + detected_point[1]**2 + detected_point[2]**2)

    if distance > 2 or distance < 0.01:
        print('nada')
    else:
        print(f'distance = {distance}')
    time.sleep(0.1)

KeyboardInterrupt: 